In [1]:
import os, sys
sys.path.insert(0, os.environ['PROJECT_PATH'])

In [2]:
import pandas as pd, requests

from config.resources import path_to

In [3]:
path_to_src = os.path.join(os.environ['DATA_PATH'], 'interim', 'augmented_meetup_locations.csv')

In [4]:
loc_df = pd.read_csv(path_to_src, encoding='latin1')

In [5]:
cvs_loc_df = loc_df[ loc_df['MSA_CODE'] == 16820].reset_index()

cvs_loc_df.head()

,index,City,Country,ZipCode,Longitude,State,Latitude,MSA_NAME,MSA_CODE
0,22700,Gladstone,USA,24553,-78.84,VA,37.55,"Charlottesville, VA Metro Area",16820.0
1,22708,Wingina,USA,24599,-78.76,VA,37.62,"Charlottesville, VA Metro Area",16820.0
2,22709,Norwood,USA,24581,-78.76,VA,37.63,"Charlottesville, VA Metro Area",16820.0
3,22711,Dillwyn,USA,23936,-78.48,VA,37.52,"Charlottesville, VA Metro Area",16820.0
4,22712,Buckingham,USA,23921,-78.62,VA,37.60,"Charlottesville, VA Metro Area",16820.0


In [6]:
fields = [ 'id', 'name', 'status', 'link', 'created', 'lat', 'lon', 'who', 'category.name', 'members',
             'last_event', 'next_event', 'meta_category.best_topics.urlkey', 'topics.urlkey' ]
lon, lat = -78.84, 37.55
optionals = [ 'past_event_count', 'last_event', 'best_topics', 'topics' ]
url_template = "https://api.meetup.com/find/groups?&sign=true&photo-host=public&lon={lon}&fields={optionals}&lat={lat}&page=20&only={fields}&key={user_key}"

In [ ]:
from __future__ import print_function
from src.data.helper_utils.meetup_scraper import fetch_paginated_data, get_df_from_nested_dicts 

group_url = url_template.format(lon=lon, lat=lat, fields=",".join(fields), 
                              optionals=",".join(optionals), user_key=os.environ['API_KEY'])
groups_data = fetch_paginated_data(group_url, None)
get_df_from_nested_dicts(groups_data, fields + optionals)

# groups_df_batches = []
# nrows, _ = cvs_loc_df.shape
# for index, row in cvs_loc_df.iterrows():
#     print('Processed {} out of {} rows'.format(index, nrows), end='\r',)
#     lon, lat = row[ ['Longitude', 'Latitude'] ]
#     group_url = url_template.format(lon=lon, lat=lat, fields=",".join(fields), 
#                               optionals=",".join(optionals), user_key=os.environ['API_KEY'])
#     groups_data = fetch_paginated_data(group_url, None)
#     groups_df_batches.append(get_df_from_nested_dicts(groups_data, fields + optionals))
    

> /home/banerjee_abhimanyu3/meetups/src/data/helper_utils/meetup_scraper.py(111)get_df_from_nested_dicts()
-> try:
(Pdb) flat_df.head()
          category.name        created        id last_event.id  \
0  Outdoors & Adventure  1456938678000  19652897     246644873   
1           Photography  1250082025000   1505179  bcclrmywqbnc   
2   Sports & Recreation  1447104138000  19106139     246023663   
3                  Tech  1407163831000  16182972     244353382   
4          Food & Drink  1421190623000  18332223     246654112   

                                     last_event.name  last_event.time  \
0       Hike on the AT with views and a plane wreck.     1.515943e+12   
1                                     Shoot Off 2017     1.514646e+12   
2                      Sense and Sensibility @ Alamo     1.515960e+12   
3  November Meetup: High Performance R on Your La...     1.510090e+12   
4                                  Sedona Taphouse!      1.516057e+12   

   last_event.utc_offset  la

In [8]:
cvs_groups_df = pd.concat(groups_df_batches)
cvs_groups_df.drop_duplicates('id')
cvs_groups_df.rename(columns=lambda x: 'latitude' if x == 'lat' else x, inplace=True)
cvs_groups_df.rename(columns=lambda x: 'longitude' if x == 'lon' else x, inplace=True)
print('Number of groups founds: ', cvs_groups_df.shape[0])

cvs_groups_df.head()

Number of groups founds:  14363


,id,name,status,link,created,latitude,longitude,who,category.name,past_event_count
0,19652897,"GetHiking! Charlottesville (Charlottesville, VA)",active,https://www.meetup.com/GetHiking-Charlottesvil...,1456938678000,38.06,-78.49,Hikers,Outdoors & Adventure,None
1,1505179,Charlottesville Photography Initiative,active,https://www.meetup.com/TheCPI/,1250082025000,38.00,-78.46,Contributors,Photography,None
2,19106139,C'Ville Social Collective,active,https://www.meetup.com/TheCSC/,1447104138000,38.04,-78.52,CSC Crew,Sports & Recreation,None
3,16182972,UVa R Users Group,active,https://www.meetup.com/UVa-R-Users-Group/,1407163831000,38.00,-78.47,UseRs,Tech,None
4,18332223,Charlottesville Foodie Meetup,active,https://www.meetup.com/Charlottesville-Foodies...,1421190623000,38.04,-78.47,Foodies,Food & Drink,None


In [9]:
from src.data.msa_mapper.map_loc_to_msa import MSAMapper

In [10]:
msa_mapper_client = MSAMapper(cvs_groups_df.fillna(''))
cvs_groups_with_msa_df = msa_mapper_client.map_data(data_type='lat-lon')


Converting lat-lon coordinates to MSA data. Please wait ..
Converting lat-lon coordinates to MSA data completed!



In [12]:
path_to_dump = os.path.join(os.environ['DATA_PATH'], 'interim', 'sample_groups_cvs.csv')
cvs_groups_with_msa_df.to_csv(path_to_dump, index='False', encoding='utf-8')

cvs_groups_with_msa_df[ cvs_groups_with_msa_df['MSA_CODE'] != '16820' ].head()
# cvs_groups_with_msa_df.head()

,index,id,name,status,link,created,latitude,longitude,who,category.name,past_event_count,MSA_NAME,MSA_CODE
8,9,9992502,Rockbridge Camera Club - RCC Photography Meetup,active,https://www.meetup.com/Rockbridge-Camera-Club-...,1377613871000,37.78,-79.44,Photographers,Photography,,None,None
12,13,2318601,Lynchburg Social Club,active,https://www.meetup.com/Lynchburg-Social-Club/,1312945563000,37.40,-79.17,Friends,Socializing,,"Lynchburg, VA Metro Area",31340
14,15,26691660,Staunton 50 and over Monthly Social,active,https://www.meetup.com/Staunton-50-and-over-Mo...,1511564350000,38.15,-79.08,Socializers,Socializing,,"Staunton-Waynesboro, VA Metro Area",44420
21,22,4394692,Shenandoah Valley Social Fun Group!,active,https://www.meetup.com/SocialFunInTheValley/,1343262988000,38.07,-78.90,Fans of Fun,Socializing,,"Staunton-Waynesboro, VA Metro Area",44420
23,24,22713359,Central Virginia Networking,active,https://www.meetup.com/CentralVANetworking/,1488410201000,37.40,-79.17,Members,Career & Business,,"Lynchburg, VA Metro Area",31340
